In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install torchsummaryX

In [5]:
import numpy as np
import os
import random
import time
from torchsummaryX import summary
from tqdm import tqdm, trange

import torch
from torch import nn
from torch.utils.data import Dataset, IterableDataset, DataLoader
from torch.nn.utils import clip_grad_norm_, clip_grad_value_

import matplotlib.pyplot as plt
from multiprocessing import Pool

Helper Functions

In [6]:
def fullprint(*args, **kwargs):
  from pprint import pprint
  import numpy
  opt = numpy.get_printoptions()
  numpy.set_printoptions(threshold=numpy.inf)
  print(*args, **kwargs)
  numpy.set_printoptions(**opt)

def greedy_ordering(player_pools):
  num_players, num_champions = player_pools.shape
  champions = np.arange(1, num_champions + 1)
  
  universe_without_self = np.array([set() for _ in range(num_players)])
  player_sets = []
  for index, pool in enumerate(player_pools * champions):
    player_pool = set(pool) - set([0])
    player_sets.append(player_pool)
    universe_without_self[list(set(list(range(num_players))) - set([index]))] |= player_pool

  player_sets_excess_size = [ len(pool - pool.intersection(universe_without_self[index])) for index, pool in enumerate(player_sets) ]

  return np.argsort(player_sets_excess_size)

def hellinger_distance_loss(y_pred, y_true):
  return (1/(2 ** 0.5)) * torch.mean(torch.sqrt(torch.sum(torch.square(torch.sqrt(y_pred) - torch.sqrt(y_true)), axis=2)))

def closeness_to_uniform(y_pred, pools):
  uniform = (pools / torch.unsqueeze(torch.sum(pools, axis=2), -1))
  return hellinger_distance_loss(y_pred, uniform)

Constants

In [20]:
N_CHAMPIONS = 152
N_EPOCHS = 1000
N_PLAYERS = 10
SHARED_POOL_SIZE = 14
BATCH_SIZE = 16
SIMULATION_ITERS = 20000
BASE_PATH = os.path.join('.', 'drive', 'My Drive')
CHECKPOINT_PATH = os.path.join(BASE_PATH, 'ARAMProbabilityFunctionModel1.pt')
DATA_TEST = os.path.join(BASE_PATH, 'data', 'test')
DATA_VAL = os.path.join(BASE_PATH, 'data', 'val')
DATA_TRAIN = os.path.join(BASE_PATH, 'data', 'train')
TESTING = True

dtype = torch.float
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Define our datasets


In [8]:
class PlayerChampionPools(IterableDataset):
  def __init__(self, n_players=N_PLAYERS, n_champions=N_CHAMPIONS, shared_pool_size=SHARED_POOL_SIZE, simulation_iters=SIMULATION_ITERS):
    self.n_players = n_players
    self.n_champions = n_champions
    self.shared_pool_size = shared_pool_size
    self.simulation_iters = simulation_iters
    self.cache_x = np.empty((0, self.n_players, self.n_champions), dtype=np.double)
    self.cache_y = np.empty((0, self.n_players, self.n_champions), dtype=np.double)

  def __iter__(self):
    return self

  def __next__(self):
    print(".", end="")
    x = PlayerChampionPools.create_player_pools(self.n_players, self.n_champions, self.shared_pool_size)
    y = PlayerChampionPools.simulate(self.simulation_iters, x)

    self.cache_x = np.append(self.cache_x, np.expand_dims(x, 0), axis=0)
    self.cache_y = np.append(self.cache_y, np.expand_dims(y, 0), axis=0)

    if (self.cache_x.shape[0] >= BATCH_SIZE):
      torch.save({
          'batch_x': self.cache_x,
          'batch_y': self.cache_y
      }, os.path.join(DATA_TRAIN, 'batch-{}'.format(random.getrandbits(128)))) 
      self.cache_x = np.empty((0, self.n_players, self.n_champions))
      self.cache_y = np.empty((0, self.n_players, self.n_champions))
    
    x_torch = torch.from_numpy(x).float().cuda()
    y_torch = torch.from_numpy(y).float().cuda()

    return (x_torch, y_torch)

  '''
  Using numpy here since torch is significantly slower (10x overhead).
  I might be using it wrong, but the simulate function took upwards of 20 seconds 
  when using torch vs. 2 seconds when using numpy.
  '''
  @staticmethod
  def create_player_pools(n_players, n_champions=N_CHAMPIONS, shared_pool_size=SHARED_POOL_SIZE):
    shared_pool = np.random.choice(n_champions, shared_pool_size)

    player_pools = np.empty(0, dtype=np.double)
    for i in range(n_players):
      player_pool = np.random.choice(n_champions, random.randint(0, n_champions))
      player_final_pool = np.zeros(n_champions)
      player_final_pool[np.concatenate((player_pool, shared_pool))] = 1
      player_pools = np.concatenate((player_pools, player_final_pool))

    player_pools = np.reshape(player_pools, (n_players, n_champions))

    return player_pools

  @staticmethod
  def simulate(iters, player_pools):
    num_champions = player_pools.shape[1]
    dists = np.zeros_like(player_pools, dtype=np.double)
    champions = np.arange(1, num_champions + 1)

    for i in range(iters):
      champions_selected = set()
      for player_index, pool in enumerate(player_pools):
        player_pool = set(pool * champions) - champions_selected - set([0])
        player_selection = random.sample(player_pool, 1)[0]
        dists[player_index, int(player_selection - 1)] += 1
        champions_selected.add(player_selection)

    return dists / np.expand_dims(np.sum(dists, axis=1), axis=-1)

class SimulatedPools(Dataset):
  def __init__(self, dataset_path):
    self.dataset = [ os.path.join(dataset_path, filename) for filename in os.listdir(dataset_path) ]

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    path = self.dataset[idx]
    datapoint = torch.load(path)

    x = torch.from_numpy(datapoint['batch_x']).squeeze().float()
    y = torch.from_numpy(datapoint['batch_y']).squeeze().float()

    return (x, y)

Define our models

In [9]:
class LSTMToLinearLayer(nn.Module):
  '''
  Input Size: (batch, n_players, n_champions)
  Output Size: (batch, n_player, n_champions)
  '''
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS, is_last_layer=False, n_lstm_layers=1, lstm_dropout=0):
    super(LSTMToLinearLayer, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.is_last_layer = is_last_layer
    self.lstm = nn.LSTM(
        input_size=self.n_champions,
        hidden_size=self.n_champions,
        num_layers=n_lstm_layers,
        batch_first=True,
        dropout=lstm_dropout
    )
    self.lin = nn.Linear(self.n_players * self.n_champions, self.n_players * self.n_champions)
    self.batchnorm = nn.BatchNorm1d(self.n_players)
    self.output_layers = nn.Sequential(
        nn.LeakyReLU(0.2),
        nn.BatchNorm1d(self.n_players)
    )
  
  def forward(self, x_input):
    x, _ = self.lstm(x_input)
    x = self.batchnorm(x)
    x = self.lin(torch.flatten(x, start_dim=1))
    x = x.view((-1, self.n_players, self.n_champions))
    if not self.is_last_layer:
      x = self.output_layers(x)
    return x

class ProbabilityFunctionModel(nn.Module):
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS):
    super(ProbabilityFunctionModel, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.network = nn.Sequential(
        LSTMToLinearLayer(is_last_layer=True),
        nn.Softmax(dim=2)
    )

  def forward(self, x_input):
    return self.network(x_input)

class ProbabilityFunctionModel2(nn.Module):
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS):
    super(ProbabilityFunctionModel2, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.network = nn.Sequential(
        LSTMToLinearLayer(n_lstm_layers=3, is_last_layer=True),
        nn.Softmax(dim=2)
    )

  def forward(self, x_input):
    return self.network(x_input)

class ProbabilityFunctionModel3(nn.Module):
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS):
    super(ProbabilityFunctionModel3, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.network = nn.Sequential(
        LSTMToLinearLayer(),
        nn.Dropout(),
        LSTMToLinearLayer(),
        nn.Dropout(),
        LSTMToLinearLayer(is_last_layer=True),
        nn.Softmax(dim=2)
    )
  
  def forward(self, x_input):
    return self.network(x_input)

class ProbabilityFunctionModel4(nn.Module):
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS):
    super(ProbabilityFunctionModel4, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.lstm = nn.LSTM(
        input_size=self.n_champions,
        hidden_size=self.n_champions,
        num_layers=1,
        batch_first=True,
        dropout=0.5
    )
    self.batchnorm = nn.BatchNorm1d(self.n_players)
    self.sequential = nn.Sequential(
        nn.Linear(self.n_players * self.n_champions,  self.n_players * self.n_champions),
        nn.LeakyReLU(0.2),
        nn.BatchNorm1d(self.n_players * self.n_champions),
        nn.Dropout(),
        nn.Linear(self.n_players * self.n_champions,  self.n_players * self.n_champions), 
        nn.LeakyReLU(0.2),
        nn.BatchNorm1d(self.n_players * self.n_champions),
    )
    self.softmax = nn.Softmax(dim=2)
  
  def forward(self, x_input):
    x, _ = self.lstm(x_input)
    x = self.batchnorm(x)
    x = self.sequential(torch.flatten(x, start_dim=1))
    x = x.view((-1, self.n_players, self.n_champions))
    x = self.softmax(x)
    return x

class ProbabilityFunctionModel5(nn.Module):
  def __init__(self, n_champions=N_CHAMPIONS, n_players=N_PLAYERS):
    super(ProbabilityFunctionModel5, self).__init__()
    self.n_champions = n_champions
    self.n_players = n_players
    self.lin1 = LSTMToLinearLayer()
    self.lin2 = LSTMToLinearLayer()
    self.lin3 = LSTMToLinearLayer()
    self.lin4 = LSTMToLinearLayer(is_last_layer=True)
    self.softmax = nn.Softmax(dim=2)
  
  def forward(self, x_input):
    x1 = self.lin1(x_input)
    x2 = self.lin2(x_input)
    x3 = self.lin3(x_input)
    
    x4 = x1 * (x2 + x3)
    x4 = self.lin4(x4)
    x4 = self.softmax(x4)
    return x4

Perform Training

In [ ]:
train_set = SimulatedPools(dataset_path=DATA_TRAIN)
train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

val_set = SimulatedPools(dataset_path=DATA_VAL)
val_dataloader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

test_set = SimulatedPools(dataset_path=DATA_TEST)
test_dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

model = ProbabilityFunctionModel5().to(device)
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.25)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', threshold=0.01, patience=4, verbose=True)
start_epoch = 0
train_losses = []
val_losses = []
test_losses = []
best_loss = None

if (os.path.exists(CHECKPOINT_PATH)):
  checkpoint = torch.load(CHECKPOINT_PATH)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  start_epoch = checkpoint['end_epoch']
  train_losses = checkpoint['train_losses']
  val_losses = checkpoint['val_losses']
  test_losses = checkpoint['test_losses']
  best_loss = min([np.mean(loss) for loss in train_losses])
  N_EPOCHS = N_EPOCHS - start_epoch

summary(model, torch.randn((BATCH_SIZE, N_PLAYERS, N_CHAMPIONS)).to(device))

torch.set_printoptions(profile="full")
for epoch in range(start_epoch + 1, start_epoch + N_EPOCHS + 1):
  start = time.time()

  print('Epoch {} of {}'.format(epoch, start_epoch + N_EPOCHS), end=" ")

  ## Training Dataset
  running_train_losses = []
  for batch_idx, batch in enumerate(train_dataloader):
    print(".", end="")
    batch_x, batch_y = batch
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    dist_pred = model(batch_x)
    loss = hellinger_distance_loss(dist_pred, batch_y)
    optimizer.zero_grad()
    loss.backward()
    clip_grad_norm_(model.parameters(), 0.05)
    clip_grad_value_(model.parameters(), 0.005)
    optimizer.step()

    running_train_losses.append(loss.item())

  train_losses.append(running_train_losses)
  avg_train_loss = np.mean(running_train_losses)
  scheduler.step(avg_train_loss)

  ## Validation Dataset
  running_val_losses = []
  for batch_idx, batch in enumerate(val_dataloader):
    batch_x, batch_y = batch
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    dist_pred = model(batch_x)
    loss = hellinger_distance_loss(dist_pred, batch_y)
    running_val_losses.append(loss.item())

  val_losses.append(running_val_losses)
  avg_val_loss = np.mean(running_val_losses)

  ## Test Dataset
  running_test_losses = []
  for batch_idx, batch in enumerate(test_dataloader):
    batch_x, batch_y = batch
    batch_x = batch_x.to(device)
    batch_y = batch_y.to(device)

    dist_pred = model(batch_x)
    loss = hellinger_distance_loss(dist_pred, batch_y)
    running_test_losses.append(loss.item())

  test_losses.append(running_test_losses)
  avg_test_loss = np.mean(running_test_losses)

  if not TESTING:
    # save lowest loss
    if (best_loss is None) or (avg_train_loss < best_loss):
      torch.save({
          'model_state_dict': model.state_dict(), 
          'optimizer_state_dict': optimizer.state_dict(),
          'end_epoch': epoch,
          'train_losses': train_losses,
          'val_losses': val_losses,
          'test_losses': test_losses
      }, CHECKPOINT_PATH)
      best_loss = avg_train_loss

    # save every couple of epochs
    if (epoch % 50 == 0):
      torch.save({
          'model_state_dict': model.state_dict(), 
          'optimizer_state_dict': optimizer.state_dict(),
          'end_epoch': epoch, 
          'train_losses': train_losses,
          'val_losses': val_losses,
          'test_losses': test_losses
      }, os.path.join(BASE_PATH, 'ARAMProbabilityFunctionModel1Epoch{}.pt'.format(epoch)))

  print(
      " Avg Loss: {}, Avg Val Loss: {}, Avg Test Loss: {}, Best Loss: {}, Time Taken: {} seconds"
      .format(avg_train_loss, avg_val_loss, avg_test_loss, best_loss, time.time() - start))
  
torch.set_printoptions(profile="default") # reset

plt.figure(1, figsize=(20, 10))
plt.plot(np.mean(train_losses, axis=1), label="Training Loss", linewidth=3)
plt.plot(np.mean(val_losses, axis=1), label="Validation Loss", linewidth=3)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.rcParams.update({'font.size': 22})
plt.show()

Simulate the problem using the neural network simulator

In [ ]:
model = ProbabilityFunctionModel().to(device)
checkpoint = torch.load('/content/drive/MyDrive/WD 0.25, Batch Size 16/ARAMProbabilityFunctionModel1.pt')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

diffs_random = []
diffs_greedy = []

for idx in range(1000000):
  if idx % 10000 == 0:
    print(idx)

  pool = PlayerChampionPools.create_player_pools(10)
  greedy_pool = torch.from_numpy(np.take(pool, greedy_ordering(pool), axis=0)).unsqueeze(0).float().to(device)
  pool = torch.from_numpy(pool).unsqueeze(0).float().to(device)

  rand_sim = model(pool)
  diffs_random.append(closeness_to_uniform(rand_sim, pool).item())

  greedy_sim = model(greedy_pool)
  diffs_greedy.append(closeness_to_uniform(greedy_sim, greedy_pool).item())

plt.figure(1, figsize=(20, 10))
plt.hist([diffs_random, diffs_greedy], label=["random", "greedy"])
plt.xlabel('Hellinger Distance to Uniform Distribution')
plt.ylabel('Number of Instances')
plt.rcParams.update({'font.size': 22})
plt.legend()
plt.show()

print("diffs random mean: ", np.mean(diffs_random))
print("diffs greedy mean: ", np.mean(diffs_greedy))

Test out various architectures

In [ ]:
model = ProbabilityFunctionModel().to(device)
model2 = ProbabilityFunctionModel2().to(device)
model3 = ProbabilityFunctionModel3().to(device)
model4 = ProbabilityFunctionModel4().to(device)
model5 = ProbabilityFunctionModel5().to(device)
optimizer = torch.optim.Adam(model.parameters())
optimizer2 = torch.optim.Adam(model2.parameters())
optimizer3 = torch.optim.Adam(model3.parameters())
optimizer4 = torch.optim.Adam(model4.parameters())
optimizer5 = torch.optim.Adam(model5.parameters())
new_losses = []
new_losses2 = []
new_losses3 = []
new_losses4 = []
new_losses5 = []

rand_input = torch.randn((BATCH_SIZE, N_PLAYERS, N_CHAMPIONS)).to(device)
summary(model, rand_input)
summary(model2, rand_input)
summary(model3, rand_input)
summary(model4, rand_input)
summary(model5, rand_input)

ds = SimulatedPools(dataset_path=DATA_VAL)
dl = DataLoader(ds, batch_size=32, shuffle=True)

for epoch in range(300):
  for idx, batch in enumerate(dl):
    (batch_x, batch_y) = batch
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    dist_pred = model(batch_x)
    loss = hellinger_distance_loss(dist_pred, batch_y)
    optimizer.zero_grad()
    loss.backward()
    clip_grad_norm_(model.parameters(), 0.05)
    clip_grad_value_(model.parameters(), 0.005)
    optimizer.step()
    new_losses.append(loss.item())

    dist_pred2 = model2(batch_x)
    loss2 = hellinger_distance_loss(dist_pred2, batch_y)
    optimizer2.zero_grad()
    loss2.backward()
    clip_grad_norm_(model2.parameters(), 0.05)
    clip_grad_value_(model2.parameters(), 0.005)
    optimizer2.step()
    new_losses2.append(loss2.item())

    dist_pred3 = model3(batch_x)
    loss3 = hellinger_distance_loss(dist_pred3, batch_y)
    optimizer3.zero_grad()
    loss3.backward()
    clip_grad_norm_(model3.parameters(), 0.05)
    clip_grad_value_(model3.parameters(), 0.005)
    optimizer3.step()
    new_losses3.append(loss3.item())

    dist_pred4 = model4(batch_x)
    loss4 = hellinger_distance_loss(dist_pred4, batch_y)
    optimizer4.zero_grad()
    loss4.backward()
    clip_grad_norm_(model4.parameters(), 0.05)
    clip_grad_value_(model4.parameters(), 0.005)
    optimizer4.step()
    new_losses4.append(loss4.item())

    dist_pred5 = model5(batch_x)
    loss5 = hellinger_distance_loss(dist_pred5, batch_y)
    optimizer5.zero_grad()
    loss5.backward()
    clip_grad_norm_(model5.parameters(), 0.05)
    clip_grad_value_(model5.parameters(), 0.005)
    optimizer5.step()
    new_losses5.append(loss5.item())

    print(
        "Epoch {}, loss1: {}, loss2: {}, loss3: {}, loss4: {}, loss5: {}"
        .format(epoch, loss.item(), loss2.item(), loss3.item(), loss4.item(), loss5.item()))

plt.figure(1)
plt.plot(new_losses, label="ARAMNet", linewidth=3)
plt.plot(new_losses2, label="Network 2", linewidth=3)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.rcParams.update({'font.size': 22})
plt.show()

plt.figure(2)
plt.plot(new_losses, label="ARAMNet", linewidth=3)
plt.plot(new_losses3, label="Network 4", linewidth=3)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.rcParams.update({'font.size': 22})
plt.show()

plt.figure(3)
plt.plot(new_losses, label="ARAMNet", linewidth=3)
plt.plot(new_losses4, label="Network 3", linewidth=3)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.rcParams.update({'font.size': 22})
plt.show()

plt.figure(4)
plt.plot(new_losses, label="ARAMNet", linewidth=3)
plt.plot(new_losses5, label="Network 5", linewidth=3)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.rcParams.update({'font.size': 22})
plt.show()

Generate Random Pools (and save them)

In [ ]:
for i in range(500):
  print("Start {} ...".format(i), end=" ")
  start = time.time()
  x = PlayerChampionPools.create_player_pools(10)
  y = PlayerChampionPools.simulate(100000, x)
  x = np.expand_dims(x, 0)
  y = np.expand_dims(y, 0)
  torch.save({ 'batch_x': x, 'batch_y': y}, os.path.join(DATA_TRAIN, 'batch-{}'.format(random.getrandbits(256))))
  print("Completed in {} seconds".format(time.time() - start))